In [19]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow

In [29]:



MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


In [30]:
client.search_experiments()

[<Experiment: artifact_location='/home/mlops_zoomcamp/notebooks/02- Experiment Tracking/mlruns/2', creation_time=1685393923973, experiment_id='2', last_update_time=1685393923973, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/home/mlops_zoomcamp/notebooks/02- Experiment Tracking/mlruns/1', creation_time=1685290772018, experiment_id='1', last_update_time=1685290772018, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1684956958338, experiment_id='0', last_update_time=1684956958338, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
client.create_experiment(name="my-cool-experiment")


'2'

In [35]:
runs = client.search_runs(
    experiment_ids="1",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"],
)

In [36]:
for r in runs:
    print(f"runid {r.info.run_id} , rmse {r.data.metrics['rmse']:.4f}" )

runid b5016e2ed4b643a98c9a64dde68bda2d , rmse 6.3028
runid a97fa0b41b794f7097794bff6e7738c9 , rmse 6.3028
runid f3939f8b4d8b40dfad811d97c336abae , rmse 6.3082
runid 4cf605bf44194f2fbf858d7303bc667f , rmse 6.3133
runid b920ad29ba7746fea0e41d30cb495ede , rmse 6.3297


In [31]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [17]:
run_id = "f3939f8b4d8b40dfad811d97c336abae"
model_uri =f"runs:/{run_id}/model"
mlflow.register_model(model_uri, name = "nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/06/01 11:13:01 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1685617981918, current_stage='None', description=None, last_updated_timestamp=1685617981918, name='nyc-taxi-regressor', run_id='f3939f8b4d8b40dfad811d97c336abae', run_link=None, source=('/home/mlops_zoomcamp/notebooks/02- Experiment '
 'Tracking/mlruns/1/f3939f8b4d8b40dfad811d97c336abae/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [25]:
model_name = "nyc-taxi-regressor"
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1685617896711, description=None, last_updated_timestamp=1685617896726, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685617896726, current_stage='None', description=None, last_updated_timestamp=1685617896726, name='nyc-taxi-experiment', run_id='f3939f8b4d8b40dfad811d97c336abae', run_link=None, source=('/home/mlops_zoomcamp/notebooks/02- Experiment '
  'Tracking/mlruns/1/f3939f8b4d8b40dfad811d97c336abae/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=1>], name='nyc-taxi-experiment', tags={}>,
 <RegisteredModel: aliases={}, creation_timestamp=1685613641362, description='The NYC taxi predictor for duration prediction', last_updated_timestamp=1685618955008, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685617981918, current_stage='Staging', description=None, last_updated_timestamp=1685618955008, name='nyc-taxi-regressor', run_id='f3939f8b4d8b40dfad811d97c336abae', ru

In [26]:
latest_versions =client.get_latest_versions(name=model_name)

In [24]:
for version in latest_versions:
    print(f"version: {version.version} stage: {version.current_stage}")

version: 3 stage: Staging


In [26]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Staging",
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685617981918, current_stage='Staging', description=None, last_updated_timestamp=1685618955008, name='nyc-taxi-regressor', run_id='f3939f8b4d8b40dfad811d97c336abae', run_link=None, source=('/home/mlops_zoomcamp/notebooks/02- Experiment '
 'Tracking/mlruns/1/f3939f8b4d8b40dfad811d97c336abae/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [27]:
from datetime import datetime

In [33]:
model_version = 1
new_stage="Staging"
date = datetime.today().date()

client.update_model_version(
name=model_name,
version = model_version,
description =f"model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685613641777, current_stage='Staging', description='model version 1 was transitioned to Staging on 2023-06-01', last_updated_timestamp=1685619672596, name='nyc-taxi-regressor', run_id='f840d8cdc2ca446f8b16c8a29be789b7', run_link='', source=('/home/mlops_zoomcamp/notebooks/02- Experiment '
 'Tracking/mlruns/1/f840d8cdc2ca446f8b16c8a29be789b7/artifacts/models_mlflow'), status='READY', status_message=None, tags={}, user_id=None, version=1>

In [10]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [11]:
import os 
os.getcwd()

'/home/mlops_zoomcamp/notebooks/02- Experiment Tracking'

In [12]:
df = read_dataframe("/home/mlops_zoomcamp/notebooks/data/green_tripdata_2021-03.parquet")


In [32]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [13]:
run_id = "f840d8cdc2ca446f8b16c8a29be789b7"
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


/tmp/ipykernel_25495/4206337130.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/home/mlops_zoomcamp/notebooks/02- Experiment Tracking/preprocessor'

In [15]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [16]:
X_test = preprocess(df, dv)


In [17]:
target = "duration"
y_test = df[target].values

In [33]:
model_name= "nyc-taxi-regressor"
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

2023/06/01 12:33:11 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[12:33:11] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 1.35 s, sys: 45.2 ms, total: 1.4 s
Wall time: 1.07 s


{'rmse': 6.308880507610186}

In [37]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)


2023/06/01 12:51:55 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[12:51:55] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 2.5 s, sys: 0 ns, total: 2.5 s
Wall time: 1.27 s


{'rmse': 6.252644516870416}

In [38]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1685613641777, current_stage='Production', description='model version 1 was transitioned to Staging on 2023-06-01', last_updated_timestamp=1685623985494, name='nyc-taxi-regressor', run_id='f840d8cdc2ca446f8b16c8a29be789b7', run_link='', source=('/home/mlops_zoomcamp/notebooks/02- Experiment '
 'Tracking/mlruns/1/f840d8cdc2ca446f8b16c8a29be789b7/artifacts/models_mlflow'), status='READY', status_message=None, tags={}, user_id=None, version=1>